<span style="font-size: 20px; font-weight: bold;">Importing necessary libraries and seeding for reproducibility of the solution</span>


In [42]:
import pandas as pd
import numpy as np
import datetime
from datetime import timedelta
import random
import copy
import time
import math
import scipy.special as sps
import plotly.express as px
import matplotlib.pyplot as plt
from pyomo.environ import *
from collections import OrderedDict
from collections import deque
random.seed(0)

<span style="font-size: 20px; font-weight: bold;">Integer Linear Program Formulation using pyomo. The function returns the selected string and list of propagated delays of the selected strings.</span>

In [43]:
def route_optimizer(stagewise_routes, prop_delay, route_strings):
    
    model = ConcreteModel()
    n = len(stagewise_routes)
    penaltyCost = 10000
    
    sflights = []
    for fs in stagewise_routes:
        for f in fs:
            if f not in sflights:
                sflights.append(f)
        
    m = len(sflights)
    nparray = []
    
    for i in sflights:
        npList=[]
        cc = 0
        for j in stagewise_routes:
            if i in j:
                npList.append(1)
            else:
                npList.append(0)
            cc+=1
        nparray.append(npList)
        
    varList = []
                    
    for i in range(n):
        varList.append(i)
        
    model.xVar = Var(varList, within = Integers, bounds = (0,1), initialize = 0)
    
    model.cover_constraint = ConstraintList()
    for i in range(m):
        a=nparray[i]
        add=0
        
        for j in range(n):
            k=a[j]*model.xVar[j]
            add=add+k
        
        model.cover_constraint.add(add <=1)

    model.airport_capacity_constraint = ConstraintList()

    model.airport_capacity_constraint.add(sum(model.xVar[i] for i in range(n)) <= 100)
    
    aircraftRouting = []
    penaltyProd = []
    
    for i in range(n):
        penaltyProd.append(penaltyCost * len(stagewise_routes[i]) * model.xVar[i])
        aircraftRouting.append(prop_delay[i] * model.xVar[i])
        
    model.z = Var(within=Reals, bounds=None, initialize=0)
    model.z_constraint = Constraint(expr=model.z == sum(aircraftRouting) - sum(penaltyProd))
    model.value = Objective(expr=model.z, sense=minimize)
        
    result_obj = SolverFactory('mindtpy').solve(model, mip_solver='glpk', nlp_solver = 'ipopt', tee = True)
    sol_routes = []
    sol_delay = []
    index = []
    stringIndex = []
    c = 0
    for v in range(n):
        if model.xVar[v].value != 0:
            sol_routes.append(model.xVar[v].value)
            sol_delay.append(prop_delay[v])
            index.append(c)
            stringIndex.append(v)
        c+=1

    return sol_routes, index, sol_delay

<span style="font-size: 20px; font-weight: bold;">Function to calculate the total propagated delay of a string.</span>

In [44]:
def delay_calculator(seq):
    prop_delay = 0.0
    counter = 1
    print(seq)
    
    while counter < len(seq):
        temp_delay = min(0, str_to_minutes(seq[counter][2]) - str_to_minutes(seq[counter-1][3]) - seq[counter-1][4])
        prop_delay = prop_delay + temp_delay
        counter+=1
        
    return abs(prop_delay)

<span style="font-size: 20px; font-weight: bold;">This block of code enumerates all the strings and then feeds them to the optimization moodel to get the optimal strings. Here the Test cases 1,2,3 and 4 can be read from the Data Folder or can be hard coded as can be seen in the commented lines.</span>

In [48]:
def str_to_minutes(time_str):
    hours, minutes = map(int, time_str.split(':'))
    return hours * 60 + minutes

def within_time_window(arrival_time_str, departure_time_str, time_window=180):
    arrival_time = str_to_minutes(arrival_time_str)
    departure_time = str_to_minutes(departure_time_str)
    return arrival_time < departure_time <= arrival_time + time_window

def build_flight_sequences(flight_legs):
    sequences = []
    queue = deque([([leg], [l for l in flight_legs if l != leg]) for leg in flight_legs])

    while queue:
        current_sequence, remaining_legs = queue.pop()

        if not remaining_legs:
            if len(current_sequence) > 1:
                sequences.append(current_sequence)
            continue

        for leg in remaining_legs:
            if (current_sequence[-1][1] == leg[0] and
                    within_time_window(current_sequence[-1][3], leg[2])):
                new_remaining_legs = remaining_legs[:]
                new_remaining_legs.remove(leg)
                new_sequence = current_sequence + [leg]
                queue.append((new_sequence, new_remaining_legs))

            # Add substring (sequence without this leg)
            substring = current_sequence[:]
            if len(substring) > 1 and substring not in sequences:
                sequences.append(substring)

    return sequences

df = pd.read_excel("Data/Test_Case_4.xlsx",index_col=False)
flight_legs = []
for i,r in df.iterrows():
    flight_legs.append(((r["ORIGIN"],r["DEST"], r["CRS_DEP_DATETIME"],r["CRS_ARR_DATETIME"], r["ARR_DELAY_NEW"]))

# flight_legs = [
#     ('BOM', 'DEL', '06:00', '08:15', 50),
#     ('DEL', 'AMD', '09:10', '10:55', 100),
#     ('AMD', 'BOM', '11:45', '13:30', 150),
#     ('BOM', 'AMD', '14:15', '16:00', 50),
#     ('AMD', 'DEL', '16:45', '18:30', 150),
#     ('DEL', 'BOM', '19:15', '21:30', 150),
#     ('BOM', 'DEL', '22:05', '11:50', 50),
#     ('DEL', 'AMD', '07:00', '08:45', 0),
#     ('AMD', 'BOM', '09:30', '11:15', 190),
#     ('BOM', 'DEL', '12:00', '14:15', 290),
# ]

# flight_legs = [
#     ('BOM', 'DEL', '05:00', '07:15', 120),
#     ('BOM', 'DEL', '06:00', '08:15', 35),
#     ('DEL', 'AMD', '09:10', '10:55', 100),
#     ('DEL', 'AMD', '08:45', '10:30', 260),
#     ('AMD', 'BOM', '11:45', '13:30', 75),
#     ('AMD', 'BOM', '12:45', '14:30', 95),
# ]

# flight_legs = [
#     ("BOM (Mumbai)", "DEL (Delhi)", "06:00", "08:15", 50),
#     ("DEL (Delhi)", "AMD (Ahmedabad)", "09:00", "10:45", 100),
#     ("AMD (Ahmedabad)", "BOM (Mumbai)", "11:30", "13:15", 150),
#     ("BOM (Mumbai)", "AMD (Ahmedabad)", "14:00", "15:45", 250),
#     ("AMD (Ahmedabad)", "DEL (Delhi)", "16:30", "18:15", 300),
#     ("DEL (Delhi)", "BOM (Mumbai)", "19:00", "21:15", 150),
#     ("BOM (Mumbai)", "DEL (Delhi)", "22:00", "23:55", 200),
#     ("DEL (Delhi)", "AMD (Ahmedabad)", "06:30", "08:15", 150),
#     ("AMD (Ahmedabad)", "BOM (Mumbai)", "09:00", "10:45", 120),
#     ("BOM (Mumbai)", "DEL (Delhi)", "11:30", "13:45", 200),
#     ("DEL (Delhi)", "AMD (Ahmedabad)", "14:30", "16:15", 150),
#     ("AMD (Ahmedabad)", "BOM (Mumbai)", "17:00", "18:45", 50),
#     ("BOM (Mumbai)", "AMD (Ahmedabad)", "19:30", "21:15", 145),
#     ("AMD (Ahmedabad)", "DEL (Delhi)", "22:00", "23:45", 250),
#     ("DEL (Delhi)", "BOM (Mumbai)", "06:00", "08:15", 150),
]

# flight_legs = [
#     ("BOM (Mumbai)", "DEL (Delhi)", "06:00", "08:15", 50),
#     ("DEL (Delhi)", "AMD (Ahmedabad)", "09:00", "10:45", 100),
#     ("AMD (Ahmedabad)", "BOM (Mumbai)", "11:30", "13:15", 150),
#     ("BOM (Mumbai)", "AMD (Ahmedabad)", "14:00", "15:45", 250),
#     ("AMD (Ahmedabad)", "DEL (Delhi)", "16:30", "18:15", 300),
#     ("DEL (Delhi)", "BOM (Mumbai)", "19:00", "21:15", 150),
#     ("BOM (Mumbai)", "DEL (Delhi)", "22:00", "23:55", 200),
#     ("DEL (Delhi)", "AMD (Ahmedabad)", "06:30", "08:15", 150),
#     ("AMD (Ahmedabad)", "BOM (Mumbai)", "09:00", "10:45", 120),
#     ("BOM (Mumbai)", "DEL (Delhi)", "11:30", "13:45", 200),
#     ("DEL (Delhi)", "AMD (Ahmedabad)", "14:30", "16:15", 150),
#     ("AMD (Ahmedabad)", "BOM (Mumbai)", "17:00", "18:45", 50),
#     ("BOM (Mumbai)", "AMD (Ahmedabad)", "19:30", "21:15", 145),
#     ("AMD (Ahmedabad)", "DEL (Delhi)", "22:00", "23:45", 250),
#     ("DEL (Delhi)", "BOM (Mumbai)", "06:00", "08:15", 150),
#     ("BOM (Mumbai)", "DEL (Delhi)", "09:00", "11:15", 250),
#     ("DEL (Delhi)", "AMD (Ahmedabad)", "12:00", "13:45", 100),
#     ("AMD (Ahmedabad)", "BOM (Mumbai)", "14:30", "16:15", 350),
#     ("BOM (Mumbai)", "AMD (Ahmedabad)", "17:00", "18:45", 140),
#     ("AMD (Ahmedabad)", "DEL (Delhi)", "19:30", "21:15", 150)
# ]

sol_routes = []
sol_routes.append(-99)

while len(sol_routes) > 0:
    flight_sequences = build_flight_sequences(flight_legs)
    flight_sequences_id = []

    for seq in flight_sequences:
        temp_seq = []
        for s in seq:
            temp_seq.append(flight_legs.index(s))
        flight_sequences_id.append(temp_seq)

    delay_list = []
    for f in flight_sequences:
        delay_list.append(delay_calculator(f))
        
    print("Printing flight legs")
    print(flight_legs)
    print("Printing delay list: ", delay_list)
    print("Printing flight_sequences: ", flight_sequences)
    sol_routes = []
    if len(flight_sequences) != 0:
        sol_routes, index, sol_delay = route_optimizer(flight_sequences_id, delay_list, flight_sequences)
    print("Printing selected indices: ", index)
    
    if len(flight_sequences) != 0:
        for i in index:
            print("===============")
            print("Selected sequences: ", flight_sequences[i])
            for fs in flight_sequences[i]:
                flight_legs.remove(fs)
    
    if len(flight_legs) == 0:
        sol_routes = []

[('BOM (Mumbai)', 'AMD (Ahmedabad)', '19:30', '21:15', 145), ('AMD (Ahmedabad)', 'DEL (Delhi)', '22:00', '23:45', 250)]
[('AMD (Ahmedabad)', 'BOM (Mumbai)', '17:00', '18:45', 50), ('BOM (Mumbai)', 'AMD (Ahmedabad)', '19:30', '21:15', 145)]
[('AMD (Ahmedabad)', 'BOM (Mumbai)', '17:00', '18:45', 50), ('BOM (Mumbai)', 'AMD (Ahmedabad)', '19:30', '21:15', 145), ('AMD (Ahmedabad)', 'DEL (Delhi)', '22:00', '23:45', 250)]
[('DEL (Delhi)', 'AMD (Ahmedabad)', '14:30', '16:15', 150), ('AMD (Ahmedabad)', 'BOM (Mumbai)', '17:00', '18:45', 50)]
[('DEL (Delhi)', 'AMD (Ahmedabad)', '14:30', '16:15', 150), ('AMD (Ahmedabad)', 'BOM (Mumbai)', '17:00', '18:45', 50), ('BOM (Mumbai)', 'AMD (Ahmedabad)', '19:30', '21:15', 145)]
[('DEL (Delhi)', 'AMD (Ahmedabad)', '14:30', '16:15', 150), ('AMD (Ahmedabad)', 'BOM (Mumbai)', '17:00', '18:45', 50), ('BOM (Mumbai)', 'AMD (Ahmedabad)', '19:30', '21:15', 145), ('AMD (Ahmedabad)', 'DEL (Delhi)', '22:00', '23:45', 250)]
[('DEL (Delhi)', 'AMD (Ahmedabad)', '14:30', 

INFO: Original model has 16 constraints (0 nonlinear) and 0 disjunctions, with
    67 variables, of which 0 are binary, 66 are integer, and 1 are continuous.
INFO: rNLP is the initial strategy being used.
INFO: Relaxed NLP: Solve relaxed integrality
INFO: Relaxed NLP: OBJ: -139365.00145104004  LB: -inf  UB: inf  TIME:0.37s
INFO: ---MindtPy main Iteration 1---
INFO: MIP 1: Solve main problem.
INFO: MIP 1: OBJ: -139365.0  LB: -139365.0  UB: inf  TIME: 0.41s
INFO: Fixed-NLP 1: Solve subproblem for fixed integers.
INFO: Fixed-NLP 1: OBJ: -139365.0  LB: -139365.0  UB: -139365.0  TIME: 0.65s
INFO: MindtPy exiting on bound convergence. LB: -139365.0 + (tol 0.0001) >=
    UB: -139365.0
Printing selected indices:  [2, 6, 25, 33, 42, 57]
Selected sequences:  [('AMD (Ahmedabad)', 'BOM (Mumbai)', '17:00', '18:45', 50), ('BOM (Mumbai)', 'AMD (Ahmedabad)', '19:30', '21:15', 145), ('AMD (Ahmedabad)', 'DEL (Delhi)', '22:00', '23:45', 250)]
Selected sequences:  [('DEL (Delhi)', 'AMD (Ahmedabad)', '14:3

In [53]:
def str_to_minutes1(time_str):
    hours, minutes = map(int, time_str.split(':'))
    return hours * 60 + minutes

def delay_calculator1(seq):
    prop_delay = 0.0
    counter = 1
    print(seq)
    
    while counter < len(seq):
        temp_delay = min(0, str_to_minutes1(seq[counter][2]) - str_to_minutes1(seq[counter-1][3]) - seq[counter-1][4])
        prop_delay = prop_delay + temp_delay
        counter+=1
        
    return abs(prop_delay)

seq = [[('BOM', 'DEL', '06:00', '08:15', 50), ('DEL', 'AMD', '09:10', '10:55', 145.15295258602873)], [('DEL', 'AMD', '07:00', '08:45', 0), ('AMD', 'BOM', '09:30', '11:15', 161.60194007597673)], [('AMD', 'BOM', '11:45', '13:30', 150), ('BOM', 'AMD', '14:15', '16:00', 147.49803720269503), ('AMD', 'DEL', '16:45', '18:30', 149.5846655605235)]]
d = []
for s in seq:
    d.append(delay_calculator1(s))
d

[('BOM', 'DEL', '06:00', '08:15', 50), ('DEL', 'AMD', '09:10', '10:55', 145.15295258602873)]
[('DEL', 'AMD', '07:00', '08:45', 0), ('AMD', 'BOM', '09:30', '11:15', 161.60194007597673)]
[('AMD', 'BOM', '11:45', '13:30', 150), ('BOM', 'AMD', '14:15', '16:00', 147.49803720269503), ('AMD', 'DEL', '16:45', '18:30', 149.5846655605235)]


[0.0, 0.0, 207.49803720269503]